<table style='border: none' align='left'>
   <tr style='border: none'>
      <th style='border: none'><font size='5' face='verdana' color='black'><b>Deploy a custom model to the Watson Machine Learning repository using scikit-learn</b></th>
      <th style='border: none'><img src='https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true' alt='Watson Machine Learning icon' height='40' width='40'></th>
   </tr>
</table>

This notebook demonstrates how to deploy a custom model that uses a non-supported framework to the `Watson Machine Learning (WML)` using a `scikit-learn estimator`. For supported model types (frameworks), refer to this <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html" target="_blank" rel="noopener no referrer">link</a>.

You will deploy the model from the <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/1835c567cd309d54fc797900f79a60f9" target="_blank" rel="noopener no referrer">Use statsmodels to forecast the Consumer Price Index (time series)</a> notebook to the `Watson Machine Learning (WML)` repository. The notebook is in the `Watson Studio` community.

You will learn how to write a `scikit-learn custom estimator`, then deploy it to the `Watson Machine Learning` repository.

The data set, <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/304ff4a1704b967dd29693a4d32ba626" target="_blank" rel="noopener no referrer">Consumer Prices</a>, which is originally sourced from the <a href="http://www.ilo.org/stat/" target="_blank" rel="noopener no referrer">International Labour Organization</a> measures the Consumer Price Index of different countries over a period of time. The **Consumer Price Index (CPI)** is defined as a measure of the average change over time in the prices paid by urban consumers for a market basket of consumer goods and services. Forecasting this value is useful because it's a valuable economic indicator used to predict the rate of inflation. It also affects decision making pertaining to income payments.

This notebook uses Python 3.6, the `statsmodels`, and the `watson-machine-learning-client` packages.

## Learning goals
- Load data as a dataframe
    - Create a time series object
- Explore data
    - Check the stationarity of the time series
        - Seasonal decomposition
        - Dicky-Fuller test
- Prepare data - stationarizing the series
- Optimize the ARIMA parameters and create the model
    - ACF and PACF plots to identify parameters
    - Use grid search for ARIMA
- Train the model
- Deploy the model
- Score the deployed model

## Contents
1. [Load data](#load)
2. [Explore data](#explore)
3. [Prepare data](#prepare)
4. [Model selection](#modelselection)
5. [Deploy the model](#deploy)
6. [Score the model](#score)
7. [Summary and next steps](#summary)

<a id="load"></a>
## 1. Load data 

In this section, you will load the time series data as a dataframe and modify it so that the index is a datetime variable.

The data set contains records of the Consumer Price Indices (CPI) of various countries over time, from 1969 to 2008.

First, install the `numpy` and `scipy` packages required for interacting with data and modeling. The `wget` package will be used to download the data set.

In [ ]:
# Install packages.
!pip install --upgrade numpy
!pip install --upgrade scipy==1.2.1
!pip install --upgrade wget

Download the [Consumer Prices](https://dataplatform.cloud.ibm.com/exchange/public/entry/view/304ff4a1704b967dd29693a4d32ba626) from the [Gallery](https://dataplatform.cloud.ibm.com/community?context=analytics) using the following code.

In [ ]:
# Download the data set.
import wget
import os

price_data = 'Consumer prices.csv' 

if not os.path.isfile(price_data):
    link_to_data = 'https://api.dataplatform.cloud.ibm.com/v2/gallery-assets/entries/304ff4a1704b967dd29693a4d32ba626/data?accessKey=d1bec8d606656afaa378d73205c1e649'
    price_data = wget.download(link_to_data)

print(price_data)

In [ ]:
# Load the data as a dataframe
import pandas as pd
import datetime

dateparse = lambda dates: datetime.datetime.strptime(dates, '%Y')
consumer_prices = pd.read_csv(price_data, parse_dates=['Year'], index_col = 'Year', date_parser=dateparse, engine='python')
consumer_prices.head()

Before the data set is loaded as a dataframe, the *Year* column is converted into a datetime type variable. Each date is unique, which allows the *Year* column to be made the index of the data set so that the dataframe can be used as a time series object.

Now, when you list the dataframe information, you can see the DatatimeIndex and the rest of the columns in the data set. The *Value* column represents the value of the country's Consumer Price Index that year.

In [ ]:
consumer_prices.tail()

In [ ]:
consumer_prices.info()

In [ ]:
# Distribution of columns.
with pd.option_context('display.max_rows', None):
    for col in consumer_prices:
        print(col, len(consumer_prices[col].value_counts()))

Import the necessary data visualization packages and plot the time series object to observe the several plotted values by country.

In [ ]:
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt
register_matplotlib_converters()

In [ ]:
# Plot the data by country.
plt.figure(figsize=(15, 40))
for country in (consumer_prices['Country or Area'].unique()[-30:-1]):    
    plt.plot(consumer_prices.loc[(consumer_prices['Country or Area'] == country)].index.to_pydatetime(), consumer_prices['Value'].loc[consumer_prices['Country or Area'] == country], label=country)
    plt.xlabel('Year')
    plt.ylabel('Consumer Price Index')
    plt.legend()

First, filter the data to obtain the CPI values for the United States. You can then observe the pattern of the values changing over time (1969-2008) in the country.

In [ ]:
# CPI values in the United States.
us_consumer_prices = consumer_prices[['Value']].loc[consumer_prices['Country or Area'] == 'United States']
us_consumer_prices = us_consumer_prices.sort_index()
us_consumer_prices.head()

In [ ]:
# Plot time series data.
import matplotlib.dates as mdates
plt.figure(figsize=(20, 10))
plt.xlabel('Year')
plt.ylabel('Consumer Price Index')
plt.plot(us_consumer_prices.index.to_pydatetime(), us_consumer_prices['Value']);

<a id="explore"></a>
## 2. Explore data

In this section, you will explore the data to learn more about the stationarity of the series and determine the steps to take for data preparation.

### 2.1 Checking stationarity of time series

Check the stationarity of the time series, since many models assume that the series is stationary. A time series is stationary when the mean, variance, and covariance of the data are constant and not dependent on time.

#### Seasonal Decomposition

You will use the `statsmodels` package to plot and model the time series data.

Decompose the time series to observe the trend and seasonality in the data.

In [ ]:
!pip install --upgrade statsmodels

In [ ]:
# Decomposition of the time series data.
import numpy
import statsmodels.api as sm
from pylab import rcParams

rcParams['figure.figsize'] =15, 10
decomposition = sm.tsa.seasonal_decompose(us_consumer_prices['Value'], model = 'additive')
fig = decomposition.plot()
plt.xlabel('Year')
plt.show()

As you can see, the trend is changing based on time, so you can infer that the series is not stationary. You can also use another method, called the Dicky-Fuller test, to confirm this observation.

#### Dicky-Fuller Test

In [ ]:
# Augmented Dicky-Fuller test.
from statsmodels.tsa.stattools import adfuller
dftest = adfuller(us_consumer_prices['Value'], autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','Number of Lags Used','Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

You can use the Dicky-Fuller test to check the stationarity of the time series data.

The null hypothesis $H_{o}$ assumes that the time series is dependent on time (that it is non-stationary). Since the `Test Statistic` is larger than the `Critical Values`, we cannot reject the null hypothesis and understand that the series is **non-stationary**.

<a id="prepare"></a>
## 3. Prepare data

### 3.1 ACF & PACF plots

A common tool used to forecast time series data is the `ARIMA` (**A**uto **R**egressive **I**ntegrated **M**oving **A**verage) model. The model has 3 parameters 
- `p` - the parameter associated with the Auto-Regressive part of the ARIMA model. You can use the **PACF (partial autocorrelation function)** plot to find the optimal value.
- `d` - the parameter associated with the Integration part of the ARIMA model. This is the **order of difference**, or the number of times the time series is differenced in order to stationarize the series.
- `q` - the parameter associated with the Moving Average part of the ARIMA model. You can use the **ACF (autocorrelation function)** plot to find the optimal value.

Here are the ACF and PACF plots of the original time series data.

In [ ]:
# Visualize the ACF and PACF plots.
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,6))
plot_acf(us_consumer_prices['Value'], ax = ax1)
plot_pacf(us_consumer_prices['Value'], ax = ax2, method='ywmle')
plt.show()

Since the series is not stationary, you will first use differencing before finding the parameters for the model.

### 3.2 Stationarizing the time series data

**Differencing** is used to remove the non-stationarity caused by the trend. The number of differences needed to remove stationarity determines the parameter `d`, for the Integration component of the ARIMA model. Run the following code to difference the data twice and plot the resulting values.

In [ ]:
# Differencing - remove non-stationarity.
plt.figure(figsize=(20, 10))
us_consumer_prices_dif = (((us_consumer_prices.diff()).dropna()).diff()).dropna()

plt.xlabel('Year')
plt.ylabel('Consumer Price Index (differenced)')
plt.plot(us_consumer_prices_dif['Value']);

The time series data has been differenced twice to remove stationarity. You can observe the trend of the modified data using the seasonal decomposition method once again.

In [ ]:
# Decomposition of stationarized time series.
rcParams['figure.figsize'] =15, 10
decomposition = sm.tsa.seasonal_decompose(us_consumer_prices_dif['Value'], model = 'additive')
fig = decomposition.plot()
plt.xlabel('Year')
plt.show()

From the plot, you can see that there is no observable trend or seasonality in the differenced time series data. Once again, you can use the Augmented Dicky-Fuller test to confirm that the stationarity of the data as well.

In [ ]:
# Augmented Dicky-Fuller test.
dftest = adfuller(us_consumer_prices_dif['Value'], autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', 'Number of Lags Used', 'Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

Since the `Test Statistic` here is less than the `Critical Values`, we can reject the null hypothesis and we can assume that the series is stationary. As you can see from the decomposition plot and the Dicky-Fuller test, the stationarity has been removed.

<a id="modelselection"></a>
## 4. Model Selection

Now, you can run the code below to plot the ACF and PACF plots of the modified stationary data.

In [ ]:
# Visualize the ACF and PACF plots of the stationarized series.
f, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 6))
plot_acf(us_consumer_prices_dif['Value'], ax = ax1)
plot_pacf(us_consumer_prices_dif['Value'], ax = ax2, method='ywmle')
plt.show()

### 4.1 Grid Search

To check the optimal values for the ARIMA parameters, you can perform grid search using the package below.

In [ ]:
# Install the pmdarima package.
!pip install --upgrade pmdarima

In [ ]:
# Import auto-arima - grid search for ARIMA.
from pmdarima.arima import auto_arima

Run the following code to compare the various combinations of the ARIMA parameters, as well as the seasonal parameters.

In [ ]:
# Perform grid search for the ARIMA model.
stepwise_model = auto_arima(us_consumer_prices[:'2000'], start_p=0, start_q=0, max_p=5, max_q=5, seasonal=False, d=2, D=0, 
                            trace=True, suppress_warnings=True, error_action='ignore', stepwise=True)
print(stepwise_model.aic())

The parameter set with the lowest `AIC` value (criteria that measures the model) is a good choice to fit the model. As you can see, the lowest `AIC` value is about 353.6, so the optimal parameters are `order = (0, 2, 1)`.

### 4.2 Install the custom scikit-learn estimator <a id="custom"></a>

Please refer to this <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-custom_libs_scikit_learn.html?audience=wdp&linkInPage=true" target="_blank" rel="noopener no referrer">link</a> regarding packaging a custom `scikit-learn custom transformer`. The same method can also be applied to packaging a custom `scikit-learn custom estimator`.

The `scikit-learn custom estimator` that wraps the `statsmodels` model looks like the following and is included in `sklearn_arima-0.1.zip` file that will be downloaded in this subsection.

Download the packaged `custom estimator` package.

In [ ]:
filename = 'sklearn_arima-0.1.zip'

if not os.path.isfile(filename):
    filename = wget.download('https://github.com/IBMDataScience/sample-notebooks/raw/master/Files/sklearn_arima-0.1.zip')
    
print(filename)

Install the downloaded `custom estimator` package that is compressed in `.zip` format using the `pip install` command.

In [ ]:
!pip install sklearn_arima-0.1.zip

### 4.3 Build the model <a id="build"></a>

Import required modules. The `SklearnArima` module is the `custom estimator` you installed locally in section [4.2 Install the custom scikit-learn estimator](#custom).

A `scikit-learn` pipeline consists of `transformer(s)` and a final `estimator`. You can check the details of the `scikit-learn` pipeline <a href="https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn_arima.arima import SklearnArima

In [ ]:
pipeline = Pipeline(
    [
        ('arima', SklearnArima(us_consumer_prices[:'2000'], order=(0, 2, 1)))
    ]
)

Train the model.

In [ ]:
pipeline.fit(us_consumer_prices[:'2000'])

Test the model.

- The first array is the out of sample forecasts.
- The second array is the standard error of the forecasts.
- The third array is a 2D array of the confidence interval for the forecast.

Refer to this <a href="https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima_model.ARIMAResults.forecast.html#statsmodels.tsa.arima_model.ARIMAResults.forecast" target="_blank" rel="noopener no referrer">link</a> on the return values of the `predict` method.

In [ ]:
pipeline.predict(us_consumer_prices[:'2000'])

### 4.4 Deploy the custom scikit-learn estimator <a id="custom_deploy"></a>

Import the `WatsonMachineLearningAPIClient` module.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

You need your `Watson Machine Learning (WML)` credentials to instantiate a `WatsonMachineLearningAPIClient` object.

In [ ]:
wml_credentials = {
    'url': 'https://ibm-watson-ml.mybluemix.net',
    'username': '***',
    'password': '***',
    'instance_id': '***'
}

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

Store the downloaded `custom package` in the Watson Machine Learning repository.

**Notes:**
- For client.runtimes.LibraryMetaNames.NAME, specify the value passed in the name parameter of the setup() function in the setup.py file.
- For client.runtimes.LibraryMetaNames.FILEPATH, specify the .zip file name of your custom package.
- The identifier of the stored package, `custom_package_uid`, is required for the next step.

In [ ]:
custom_estimator_meta = {
    client.runtimes.LibraryMetaNames.NAME: 'sklearn_arima',
    client.runtimes.LibraryMetaNames.DESCRIPTION: 'ARIMA model for sklearn pipeline',
    client.runtimes.LibraryMetaNames.FILEPATH: 'sklearn_arima-0.1.zip',
    client.runtimes.LibraryMetaNames.VERSION: '0.1',
    client.runtimes.LibraryMetaNames.PLATFORM: {'name': 'python', 'versions': ['3.6']}
}
custom_estimator_details = client.runtimes.store_library(custom_estimator_meta)
custom_estimator_uid = client.runtimes.get_library_uid(custom_estimator_details)
print('Custom Estimator UID: {}'.format(custom_estimator_uid))

Check the details of the `custom package` metadata.

In [ ]:
custom_estimator_details

Create a `runtime resource` object that references your stored custom package.

In [ ]:
runtimes_meta = {
    client.runtimes.ConfigurationMetaNames.NAME: 'sklearn_arima_model', 
    client.runtimes.ConfigurationMetaNames.DESCRIPTION: 'scikit-learn ARIMA model', 
    client.runtimes.ConfigurationMetaNames.PLATFORM: {'name': 'python', 'version': '3.6'}, 
    client.runtimes.ConfigurationMetaNames.LIBRARIES_UIDS: [custom_estimator_uid]
}

Store the `runtime resource` object in the Watson Machine Learning (WML) repository.

In [ ]:
runtime_details = client.runtimes.store(runtimes_meta)
runtime_details

You need the identifier of the runtime resource object, `custom_runtime_uid`, for the next step. 

In [ ]:
runtime_url = client.runtimes.get_url(runtime_details)
runtime_uid = client.runtimes.get_uid(runtime_details)
print('Runtimes URL: {}'.format(runtime_url))
print('Runtimes UID: {}'.format(runtime_uid))

Store your trained model in the Watson Machine Learning (WML) repository referencing your stored runtime resource in meta data.

In [ ]:
model_props = {
    client.repository.ModelMetaNames.NAME: 'Custom ARIMA estimator for sklearn pipeline',
    client.repository.ModelMetaNames.RUNTIME_UID: runtime_uid
}

In [ ]:
published_model = client.repository.store_model(model=pipeline, meta_props=model_props)

Print the details of the trained model.

In [ ]:
import json

In [ ]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

### 4.5 Forecasting <a id="forecast"></a>

Now, that you've built the model, you can use it to predict the Consumer Price Index for years 2001-08 and compare the predictions with the observed numbers. Run the code below to plot the observed values alongside the predicted values with a 95% confidence interval.

In [ ]:
ax = us_consumer_prices['1969':'2008'].plot(label='Observed')
forecast, stderr, conf_int = pipeline.predict(us_consumer_prices[:'2000'])
forecast = pd.DataFrame(forecast).set_index(us_consumer_prices['2001':].index)
forecast.plot(ax=ax, style = 'r', label='Forecast')

# Calculate and visualize the 95% confidence interval.
ax.fill_between(us_consumer_prices['2001':].index, conf_int[:,0], conf_int[:,1], color='dimgray', alpha=0.1)

# Add the labels to the plot.
plt.legend(('Observed', 'Forecast'))
plt.xlabel('Year')
plt.ylabel('Consumer Price Index');

In the plot above, you can observe how the predicted values (in red) measure up to the observed values (in blue) during the years 2001-08. As you can see, the time series model built in this notebook does a good job of predicting the Consumer Price Index in the United States close to the observed numbers.

## 5. Deploy the model <a id="deploy"></a>

Deploy the model to the Watson Machine Learning repository.

In [ ]:
sklearn_arima_deployment = client.deployments.create(published_model_uid, name='sklearn_pipeline_arima')

## 6. Score the model <a id="score"></a>

### 6.1 Scoring

Get the url of the `scoring endpoint`.

In [ ]:
scoring_endpoint = client.deployments.get_scoring_url(sklearn_arima_deployment)
print(scoring_endpoint)

The payload that is passed to the `scoring endpoint` is the same as the one used for testing the trained model in section [4.3 Build the model](#build).

In [ ]:
us_consumer_prices[:'2000'].values.reshape(1, -1).tolist()

In [ ]:
scoring_payload = {
    'values': us_consumer_prices[:'2000'].values.reshape(1, -1).tolist()
}

Get the prediction results of the deployed model through the `scoring endpoint`.

In [ ]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)

In [ ]:
predictions['values']

### 6.2 Forecasting

Plot the same graph in section [4.4 Forecasting](#forecast) to see if the scored results are the same as the one predicted locally.

Prepare data to plot the forecast results.

In [ ]:
forecast = numpy.array(predictions['values'][0][0])
conf_int = numpy.array(predictions['values'][2][0])
print(forecast)
print(conf_int)

In [ ]:
ax = us_consumer_prices['1969':'2008'].plot(label='Observed')
forecast = pd.DataFrame(forecast).set_index(us_consumer_prices['2001':].index)
forecast.plot(ax=ax, style = 'r', label='Forecast')

# Calculate and visualize the 95% confidence interval.
ax.fill_between(us_consumer_prices['2001':].index, conf_int[:,0], conf_int[:,1], color='dimgray', alpha=0.1)

# Add the labels to the plot.
plt.legend(('Observed', 'Forecast'))
plt.xlabel('Year')
plt.ylabel('Consumer Price Index');

The generated graph and the one in section [4.4 Forecasting](#forecast) are the same.

<a id="summary"></a>
## 7. Summary and next steps

You successfully completed this notebook and learned how to create a Time Series model. You can now analyze a time series data to check its stationarity, stationarize the series if necessary, and find the optimal parameters for the ARIMA model. You've learned how to use this model to forecast data with a confidence interval.

You also learned how to deploy a non-supported model using a `custom scikit-learn estimator`.


Check out our <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">Online Documentation</a> for more samples, tutorials, documentation, how-tos, and blog posts.

### Data citations

UNData: Consumer prices, general indices (2000=100). (2010). Retrieved from [http://data.un.org/](http://data.un.org/Data.aspx?d=LABORSTA&f=tableCode%3a7A).

Consumer Price Index: U.S. Bureau Of Labor Statistics. Retrieved from https://www.bls.gov/cpi/.



### Author

**Ananya Kaushik** is a Data Scientist at IBM.  
**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who strives to make data science easy for everyone through Watson Studio.

Copyright © 2019 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style='background:#F5F7FA; height:110px; padding: 2em; font-size:14px;'>
<span style='font-size:18px;color:#152935;'>Love this notebook? </span>
<span style='font-size:15px;color:#152935;float:right;margin-right:40px;'>Don't have an account yet?</span><br>
<span style='color:#5A6872;'>Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style='border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;'><a href='https://ibm.co/wsnotebooks' target='_blank' style='color: #3d70b2;text-decoration: none;'>Sign Up</a></span><br>
</div>